In [54]:
import yfinance as yf
import pandas as pd
# yfinance pre installed

WFC_DF = yf.download('WFC', start = '2018-4-29',end='2022-04-29')
WFC_ticker = yf.Ticker('WFC')

BAC_DF = yf.download('BAC', start = '2018-4-29',end='2022-04-29')
BAC_ticker = yf.Ticker('BAC')

# Dates are automtically set as the index.

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [56]:
# Exporting the data and importing it to provide offline access
# WFC_DF.to_csv('C:/Users/IanCH/OneDrive/WFC.csv', sep=str(','))
# BAC_DF.to_csv('C:/Users/IanCH/OneDrive/BAC.csv', sep=str(','))
WFC_DF = pd.read_csv('C:/Users/IanCH/OneDrive/WFC.csv',index_col='Date', parse_dates =[0])
BAC_DF = pd.read_csv('C:/Users/IanCH/OneDrive/BAC.csv',index_col='Date', parse_dates =[0])

In [3]:
# Calculating daily returns with adjusted close.

def Daily_R(Open, Close):
    Daily_return = Open - Close
    return round(Daily_return,5)

WFC_DF['DR'] = WFC_DF[
    ['Open','Adj Close']].apply(
    lambda df: Daily_R(
        df['Open'],df['Adj Close']), axis=1)

BAC_DF['DR'] = BAC_DF[
    ['Open','Adj Close']].apply(
    lambda df: Daily_R(
        df['Open'],df['Adj Close']), axis=1)

# axis 1 means rows.
#.apply applies the function across the selected dataframe columns.
#lambda is a small one time function, here is is required.

In [15]:
print(WFC_DF.head())
print(BAC_DF.head())

                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2018-04-30  52.560001  53.049999  51.959999  51.959999  44.958458  17050500
2018-05-01  51.759998  52.599998  51.599998  52.560001  45.477612  13903800
2018-05-02  52.380001  52.869999  52.060001  52.189999  45.157471  16397200
2018-05-03  51.619999  51.790001  50.810001  51.540001  44.930813  19216100
2018-05-04  51.169998  52.900002  51.070000  52.410000  45.689240  16660500
                 Open       High        Low      Close  Adj Close     Volume
Date                                                                        
2018-04-30  30.270000  30.430000  29.920000  29.920000  26.750227   55153900
2018-05-01  29.920000  29.959999  29.620001  29.950001  26.777048   52871700
2018-05-02  29.950001  30.070000  29.559999  29.580000  26.446249   68234600
2018-05-03  29.510000  29.520000  28.430000  29.200001  26.106506  110672700
2018-0

In [ ]:
# 3 day average
# rolling standard deviation 
# Daily & Annual Sharpie Ratio
# Daily & Annual Sortino Ratio